In [ ]:
import csv
from pandas import read_csv
from matplotlib import pyplot as plt
import numpy as np
import os
import sys
import seaborn as sns
import pandas as pd
import math
from numpy import split
import random
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import numpy
from tqdm import tqdm_notebook
from keras.callbacks import EarlyStopping
from numpy import array
#from itertools import product
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras import optimizers
from tensorflow import keras
import tensorflow as tf
from numpy.random import seed
from scipy import stats
from IPython.display import clear_output 
import statistics
import keras.backend as K

plt.rcParams['savefig.dpi'] = 300



# evaluation mertic (symmteric mean absolute percentage error)
def smape(yTrue,yPred):
  smape=0
  
  for i in range(len(yTrue)):
    smape+= abs(yTrue[i]-yPred[i])/ (abs(yTrue[i])+abs(yPred[i]))
  smape/=len(yTrue)

  return smape


def maximum(a, b):
      
    if a >= b:
        return a
    else:
        return b

def minimum(a, b):
      
    if a <= b:
        return a
    else:
        return b



def exponential_smoothing(series, alpha):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result
  
def plot_exponential_smoothing(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing - "+attack)
    plt.grid(True);

def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result

def plot_double_exponential_smoothing(series, alphas, betas):
     
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        for beta in betas:
            plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(series.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing - "+attack)
    plt.grid(True)
    plt.show()
    


#stationarity test
def tsplot(y, lags=None, figsize=(12, 7), syle='bmh'):
    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style='bmh'):
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title(attack_title+' - Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
        plt.show()
        
 
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=1):
    # flatten data
    data = train #modified

    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)
 
#Bayesian LSTM
class BayesianLSTM(LSTM): #inherits LSTM but set training=True which keeps dropout on during test time.
  def call(self, inputs):
    return super().call(inputs, training=True)

# train the model using Bayesian LSTM
def build_model(train, test, n_input, layer, unit, epoch, lr,rdo):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    test_x, test_y=to_supervised(test,n_input)
 
    # define parameters
    verbose, epochs, batch_size = 0,epoch, 8

    model = Sequential()
    
    if layer==1:
        model.add(BayesianLSTM(unit[0], input_shape=(n_input, 1),recurrent_dropout=rdo, activation = 'relu'))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True, recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
    elif layer==2:
        model.add(BayesianLSTM(unit[0], activation = 'relu', input_shape=(n_input, 1),return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu' ,recurrent_dropout=rdo))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[1], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
    elif layer==3:
        model.add(BayesianLSTM(unit[0], activation = 'relu', input_shape=(n_input, 1),return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu' ,return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[2], activation = 'relu' ,recurrent_dropout=rdo))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[2], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
          

    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='mse', metrics=['mape']) #compile the model with mean square error as the loss function
    
    #if the loss does not improve after 30 epochs, stop anyway (early stopping)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    # fit network
    train_history= model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_x, test_y),verbose=verbose,callbacks=[es])
    
    loss = train_history.history['loss']
    val_loss = train_history.history['val_loss']

    return model
 
 
# evaluate a single model
def evaluate_model(train, test, n_input,  alpha, beta,lag,layer, unit, lr, epoch,rdo, attack,label,seed):
    
  n_iterations=10

  # fit model
  model = build_model(train, test, n_input, layer, unit, epoch, lr,rdo)
  # history is a list of n_input size data

  predictions_2d = list()

  #Bayesian LSTM performs multiple iterations to create a prediction distribution (i.e., mean and variance)
  for r in range(0,n_iterations):
    predictions = list()

    #look back in test data
    history=test[0:n_input,]

    #sliding window prediction (3 years in advance=36 iterations)
    for z in test[n_input:,0]:

        y_hat=model.predict(history.reshape(1,history.shape[0],history.shape[1]))
        history=numpy.append(history, y_hat[0][0])
        history=history.reshape(history.shape[0],1)
        history=history[1:,]#move the sliding window 1 step
        predictions.append(y_hat[0][0])
    predictions_2d.append(predictions)



  predictions_2d = array(predictions_2d)
  mean_predictions=numpy.mean(predictions_2d,axis=0) #mean 
  var_predictions=numpy.var(predictions_2d,axis=0) #variance
  std_predictions=numpy.std(predictions_2d,axis=0) #standard deviation
  z=1.96
  confidence_95=z*std_predictions/math.sqrt(len(mean_predictions))# 95% confidence interval

  #calculate the height of data on the plot (plot range) to disable variance/95% conf.interval plot if too large or too small.
  upper=maximum(numpy.amax(mean_predictions),numpy.amax(test[n_input:,0]))
  lower=minimum(numpy.amin(mean_predictions),numpy.amin(test[n_input:,0]))
  p_range=abs(upper-lower)
  var_plot=True
  max_var=0
  for v in var_predictions:
    if v>p_range:
      print('disabling variance plot due to large variance=',v)
      var_plot=False
    if v>max_var:
      max_var=v
  if max_var*30<p_range:
    print('disabling variance plot due to small max variance=',max_var)
    var_plot=False

  conf_plot=True
  max_conf=0
  for conf in confidence_95:
    if conf>p_range:
      print('disabling confidence plot due to large confidence=',conf)
      conf_plot=False
    if conf>max_conf:
      max_conf=conf
  if max_conf*30<p_range:
    print('disabling confidence plot due to small max confidence=',max_conf)
    conf_plot=False


  actual=test[n_input:,]
  mape= smape(actual,mean_predictions) #calculate the error as the symmteric mean absolute percentage error (SMAPE)

  #penalizing small range (M-SMAPE)
  range_a= numpy.amax(actual)-numpy.amin(actual)
  range_p= numpy.amax(mean_predictions)-numpy.amin(mean_predictions)
  if(range_p<range_a/3):
    mape+=0.3
    print('**** PENALIZED: +0.3 ******')
  
  #record hyperparameters
  hp=[alpha, beta,lag,layer, unit, lr, epoch,rdo,label]
  print('Hyperparams=',hp)
  print('MAPE=',mape)
 
  
  #create a plot of actual vs predicted
  months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
  M=[]
  for year in range (11,23):   
      for month in months:
        if year==11 and month not in ['Jul','Aug','Sep','Oct','Nov','Dec']:
          continue
        if year==22 and month not in ['Jan','Feb','Mar']:
          continue;
        M.append(month+'-'+str(year))       

  
  M=M[-len(actual):]
  M2=[]
  p=[]
  for index,value in enumerate(M):
    if 'Dec' in M[index] or 'Mar' in M[index] or 'Jun' in M[index] or 'Sep' in M[index]:
      M2.append(M[index])
      p.append(index+1)

    
  x=range(1,len(mean_predictions)+1)
  pyplot.plot(x,actual,'b-',label='Actual')
  pyplot.plot(x,mean_predictions,'--', color="orange",label='Predicted')
  if conf_plot:
    pyplot.fill_between(x,(mean_predictions.reshape(-1)- (confidence_95.reshape(-1))), (mean_predictions.reshape(-1)+confidence_95.reshape(-1)), color='green', alpha=0.2,label='95% Confidence')
  if var_plot:
    pyplot.fill_between(x,(mean_predictions.reshape(-1)- (var_predictions.reshape(-1))), (mean_predictions.reshape(-1)+var_predictions.reshape(-1)), color='green', alpha=0.1,label='Variance')
  plt.legend(loc="best",prop={'size': 11})
  plt.axis('tight')
  plt.grid(True)
  pyplot.title(attack+' (U)', y=1.03,fontsize=18)
  #pyplot.xlabel("Month")
  pyplot.ylabel("No of incidents",fontsize=15)
  locs, labs = plt.xticks() 
  plt.xticks(ticks = p ,labels = M2, rotation='vertical',fontsize=13) 
  plt.yticks(fontsize=13)
  fig = pyplot.gcf()

  #save the results (plot and hyperparameter values) if the error is below certain threshold. 
  if mape<=0.15:  
    images_dir = 'output_validation_results/'
    plt.savefig(f"{images_dir}/"+attack.replace('/','_')+'_t_'+str(seed)+".png", bbox_inches = "tight")
    with open('output_validation_results/'+attack_title.replace('/','_')+'_t_'+str(seed)+'.txt', 'w') as f: #the name of the file includes the seed
      f.write('MAPE: '+str(mape))
      f.write('\nHyperparams: '+str(hp))
      f.close()


  pyplot.show()


  return model,mape


#forecasts the future trend of a single attack 3 years in advance
def forecast(data, e_data, model, min_list,seed, attack_label):
    

    #get best model's hyperparameters
    n_input=min_list[2]
    layer=min_list[3]
    unit=min_list[4]
    lr=min_list[5]
    epoch=min_list[6]
    rdo=min_list[7]

    #build the model using the best hyperparameters using the whole data
    model = build_model(e_data, e_data, n_input, layer, unit, epoch, lr,rdo)


    #Bayesian LSTM performs multiple iterations to create a prediction distribution (i.e., mean and variance)
    n_iterations=10
    predictions_2d=list()
    for r in range(0,n_iterations):

      predictions = list()
      predictions.append(e_data[-1,0]) #for plotting purpose
      #look back in test data
      history=e_data[-n_input:,]

      #predicts 36 months using sliding window
      for i in range(36): 
        f_hat=model.predict(history.reshape(1,history.shape[0],history.shape[1]))
        history=numpy.append(history, f_hat[0][0])
        history=history.reshape(history.shape[0],1)
        history=history[1:,]#move the sliding window 1 step
        predictions.append(f_hat[0][0][0])
      predictions_2d.append(predictions)

    predictions_2d = array(predictions_2d)
    predictions_mean=numpy.mean(predictions_2d,axis=0) #mean
    predictions_var=numpy.var(predictions_2d,axis=0) #variance
    std_predictions=numpy.std(predictions_2d,axis=0) #standard deviation
    z=1.96
    confidence_95=z*std_predictions/math.sqrt(len(predictions_mean))# 95% confidence interval

    #calculate the height of data on the plot (plot range) to disable variance/95% conf.interval plot if too large or too small.
    upper=maximum(numpy.amax(predictions_mean),numpy.amax(data))
    lower=minimum(numpy.amin(predictions_mean),numpy.amin(data))
    p_range=upper-lower
    var_plot=True
    max_var=0
    for v in predictions_var:
      if v>p_range:
        var_plot=False
      if v>max_var:
        max_var=v
    if max_var*30<p_range:
      var_plot=False

    conf_plot=True
    max_conf=0
    for conf in confidence_95:
      if conf>p_range:
        conf_plot=False
      if conf>max_conf:
        max_conf=conf
    if max_conf*30<p_range:
      conf_plot=False 


    print(e_data)
    print(predictions_mean)
    
    #Plot the forecast
    x=['2012', '2013','2014', '2015', '2016', '2017', '2018','2019', '2020', '2021','2022','2023','2024','2025']
    pyplot.plot(range(len(data)),data,'--', color="blue",label='Data')
    pyplot.plot(range(len(data), len(data)+len(predictions_mean)),predictions_mean,'--', color="red",label='Prediction')
    if conf_plot:
      pyplot.fill_between(range(len(data), len(data)+len(predictions_mean)),(predictions_mean.reshape(-1)- (confidence_95.reshape(-1))), (predictions_mean.reshape(-1)+confidence_95.reshape(-1)), color='green', alpha=0.2,label='95% Confidence')
    if var_plot:
      pyplot.fill_between(range(len(data), len(data)+len(predictions_mean)),(predictions_mean.reshape(-1)- (predictions_var.reshape(-1))), (predictions_mean.reshape(-1)+predictions_var.reshape(-1)), color='green', alpha=0.1,label='Variance')
    plt.xticks([6,18,30,42,54,66,78,90,102,114,126,138,150,162], x,fontsize=13) # positions of years on x axis

    plt.ylabel("No of incidents",fontsize=15)
    plt.legend(loc="best",prop={'size': 11})
    plt.axis('tight')
    plt.grid(True)
    pyplot.xticks(rotation=90)
    pyplot.title(attack_label+' (U)', y=1.03,fontsize=18)
    locs, labs = plt.xticks() 
    plt.yticks(fontsize=13)
     
    #save the forecast plot to file
    images_dir = 'output_hyperparameters/'
    plt.savefig(f"{images_dir}/"+attack_label.replace('/','_')+str(seed)+".png", bbox_inches = "tight")#the name of the file includes the seed

    pyplot.show()




#number of iterations of random search for hp optimisation
iterations=60
#read input data
data = read_csv('input_data/T-HMGDN-F-0711-0322.csv', header=0)
attacks=data.columns.values.tolist()
raw_values = data.values
avg_data=0


#name of attack columns (in the dataset). All refers to the number of attacks in all countries combined 
attacks=['DDoS-ALL', 'Phishing-ALL', 'Ransomware-ALL', 'Password Attack-ALL', 'SQL Injection-ALL', 'Account Hijacking-ALL', 
         'Defacement-ALL', 'Trojan-ALL', 'Vulnerability-ALL', 'Zero-day-ALL', 'Malware-ALL', 'Advanced persistent threat-ALL', 
         'XSS-ALL', 'Data Breach-ALL', 'Disinformation/Misinformation-ALL', 'Targeted Attack-ALL','Adware-ALL',
         'Brute Force Attack-ALL', 'Malvertising-ALL', 'Backdoor-ALL', 'Botnet-ALL', 'Cryptojacking-ALL',
         'Worms-ALL', 'Spyware-ALL', 'Mentions-MITM', 'Mentions-DNS Spoofing', 
         'Mentions-WannaCry Ransomware','Mentions-Dropper', 'Mentions-Wiper', 'Mentions-Pharming', 'Mentions-Insider Threat',
         'Mentions-Drive-by', 'Mentions-Rootkit', 'Mentions-Adversarial Attack', 'Mentions-Data Poisoning', 'Mentions-Deepfake',
         'Mentions-Supply Chain', 'Mentions-IoT Device Attack', 'Mentions-Keylogger', 'Mentions-DNS Tunneling', 'Mentions-Session Hijacking',
         'Mentions-URL manipulation']
done=[] #can be manually appended with the completed attacks (to avoid repeating previously optimised attacks)


#iterate over all attacks to optimise the model of each
for attack in attacks:
  clear_output()
  if attack in done:
    continue
  if not 'ALL' in attack and not 'Mentions' in attack:
    continue;
  if 'Mentions-' in attack:
    attack_title=attack[attack.index('-')+1:]
  else:
    attack_title=attack[0:attack.rindex('-')]


  if len(attack_title)>18:
    attack_title=attack_title.rsplit(' ', 1)[0]

 
  best_data=None
  best_e_data=None
   

  min_mape=999999999
  min_list=[]
  model=None
  c=1 #seed


  #alpha and beta are smoothing constants and part of hyperparameters optimisation 
  alphas_=[0.05,0.2,0.5,0.7,1]
  betas_=[0.3,0.5,0.7,1]
  #random search for hyperparameters
  for _ in range(iterations):
    alphas=[alphas_[random.randint(0, len(alphas_)-1)]]
    betas=[betas_[random.randint(0, len(betas_)-1)]]
    for alpha in alphas:
      for beta in betas:

        if alpha==1 and not beta==1:
          continue;

        #1. smoothing    
        #if alpha and beta <1 then the method is double exponential smoothing
        # if alpha<1 and beta = 1, the method is exponential smoothing
        # if both beta and alpha equal 1, then no smoothing
        if not alpha==1 and not beta==1:
            smoothed=double_exponential_smoothing(data[attack], alpha,beta)
            smoothed=smoothed[0:-1] #remove the one future step forecast from des
            plt.plot(smoothed, label="Alpha {}, beta {}".format(alpha, beta))
            plt.title("Double Exponential Smoothing - "+attack_title)
            avg_data=sum(smoothed)/len(smoothed)
        elif not alpha==1 and beta==1: 
            smoothed=exponential_smoothing(data[attack], alpha)
            plt.plot(smoothed, label="Alpha {}".format(alpha))
            plt.title("Exponential Smoothing - "+attack_title)
            avg_data=sum(smoothed)/len(smoothed)
        elif alpha==1 and beta==1:
            #no smoothing
            smoothed=data[attack]
            avg_data=smoothed.mean()

        #visualise data
        plt.plot(data[attack], label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.grid(True)
        plt.xlabel("Month")
        plt.ylabel("No of incidents")
        plt.show()
        print('average data is:',avg_data)


        #2. stationarity test
        tsplot(smoothed, lags=30)


        smoothed_all=numpy.array([smoothed]).reshape(-1,1)
        prepared_data=smoothed
        scaled_data=numpy.array(prepared_data).reshape(-1,1)

        #when did incidents start appearing? cut the part of the data earlier than that
        e_index=0 #emerging index
        for i in range(len(scaled_data)-2):
          if scaled_data[i,0]>0 and scaled_data[i+1,0]>0 and scaled_data[i+2,0]>0:
            e_index=i
            break
        print('Emerging Index=',e_index)
        scaled_e_data=scaled_data[e_index:]
        print('Length of data after cut=',len(scaled_e_data))
      

        #Model Hyperparameters
        lags_=list(range(1,maximum(len(scaled_e_data)//10,1)+1))#range of possible lags to choose from is 1 to length of data/10
        lag=lags_[random.randint(0, len(lags_)-1)]
        lrs_=[0.001,0.0001,0.0006,0.01] #learning rate
        lr=lrs_[random.randint(0, len(lrs_)-1)]
        epochs_=[30,50,100,200]
        epoch=epochs_[random.randint(0, len(epochs_)-1)]
        units_=[[50],[25],[100],[200],[64,28],[40,20],[100,50]]
        unit=units_[random.randint(0, len(units_)-1)]
        layer=len(unit)
        rdos_=[0.2,0.3,0.4,0.5] #recurrent dropout
        rdo=rdos_[random.randint(0, len(rdos_)-1)]


        repeat=3 #repeat the test for the same set of hyperparameters but with different seeds (to take the average) 
        avg_mape=0.0  #average error
        n_input=lag 
        #repeat the test for the same set of hyperparameters but with different seeds (to take the average)
        for _ in range(0,repeat):
            seed(c)
            tf.random.set_seed(c)   
            diff_label='not differenced' #curve not differenced in this experiment
            print('seed:',c,'------> DOING:',alpha, beta,lag,layer, unit, lr, epoch,rdo,diff_label)
            c+=1

            half=len(scaled_e_data)//2
            if half>36+n_input:
              split_position=len(scaled_e_data)-(36+n_input)# 3 years + # months look back testing  
            else:
              split_position=half # if data is small, take half of it (e.g. emerging attacks)

            train=scaled_e_data[0:split_position]
            validate=scaled_e_data[split_position:]
            test=scaled_e_data[split_position:]

            print('length of train=',len(train))
            print('length of validate=',len(validate))
            print('length of test=',len(test))
            train=array(train)
            validate=array(validate)
            test=array(test)

            # evaluate model and get scores
            model,mape  = evaluate_model(train,validate, n_input, alpha, beta,lag,layer, unit, lr, epoch,rdo, attack_title,diff_label,c-1)
            avg_mape=avg_mape+mape
            print ('MIN MAPE:',min_mape);
            print ('Best Hyperparams:',min_list);

        avg_mape/=repeat
        #keep record of the lowest error and the best hyperparameters
        if(avg_mape<min_mape):
          min_mape=avg_mape
          min_list=[alpha, beta,lag,layer, unit, lr, epoch,rdo,diff_label]
          best_data=scaled_data
          best_e_data=scaled_e_data
        print ('MIN MAPE:',min_mape);
        print ('Best Hyperparams:',min_list);


  #save lowest error and best hyperparameters to file
  with open('output_hyperparameters/'+attack_title.replace('/','_')+'.txt', 'w') as f:
      f.write('MAPE: '+str(min_mape))
      f.write('\nHyperparams: '+str(min_list))
      f.close()

  #optional forecast for the future trend using the best hyperparameters found for this attack type (disabled here)
  forecast_it=False
  if forecast_it:
    for a in range(1,4):
      print('seed:',a)
      seed(a)
      tf.random.set_seed(a)   
      forecast(best_data,best_e_data, model,min_list,a, attack_title)


